In [2]:
import os
import numpy as np
import pandas as pd

data_filename = "./Data/futmexico_ap2022_titulos.csv"

In [3]:
results = pd.read_csv(data_filename)
results.iloc[:8]

,Anio,Torneo,Jornada,Fecha,Lugar,Local,GolesLocal,GolesVisitante,Visitante
0,2022,apertura_mexico,1,23 Jul 21 a las 04:00,Estadio Corregidora,Querétaro,0,0,América
1,2022,apertura_mexico,1,24 Jul 21 a las 02:00,Estadio Victoria de Aguascalientes,Necaxa,0,3,Santos Laguna
2,2022,apertura_mexico,1,24 Jul 21 a las 04:00,Estadio Olímpico Benito Juárez,FC Juárez,1,3,Toluca
3,2022,apertura_mexico,1,24 Jul 21 a las 23:00,Estadio Miguel Hidalgo,Pachuca,4,0,León
4,2022,apertura_mexico,1,25 Jul 21 a las 04:00,Estadio Akron,Chivas Guadalajara,1,2,Atl. San Luis
5,2022,apertura_mexico,1,25 Jul 21 a las 19:00,Olímpico de Universitario,Pumas UNAM,0,0,Atlas Guadalajara
6,2022,apertura_mexico,1,26 Jul 21 a las 02:06,BBVA Bancomer,Monterrey,1,1,Puebla
7,2022,apertura_mexico,1,26 Jul 21 a las 04:06,Estadio Caliente,Tijuana,1,2,Tigres UANL


In [4]:
# Don't read the first row, as it is blank, and parse the date column as a date
results = pd.read_csv(data_filename, parse_dates=["Fecha"], skiprows=0)
# Fix the name of the columns
#results.columns = ["Date","Start(ET)", "Visitor Team", "VisitorPts", "Home Team", "HomePts", "Score Type", "OT?", "Attend" ,"Arena","Notes"]
results.iloc[:5]

,Anio,Torneo,Jornada,Fecha,Lugar,Local,GolesLocal,GolesVisitante,Visitante
0,2022,apertura_mexico,1,23 Jul 21 a las 04:00,Estadio Corregidora,Querétaro,0,0,América
1,2022,apertura_mexico,1,24 Jul 21 a las 02:00,Estadio Victoria de Aguascalientes,Necaxa,0,3,Santos Laguna
2,2022,apertura_mexico,1,24 Jul 21 a las 04:00,Estadio Olímpico Benito Juárez,FC Juárez,1,3,Toluca
3,2022,apertura_mexico,1,24 Jul 21 a las 23:00,Estadio Miguel Hidalgo,Pachuca,4,0,León
4,2022,apertura_mexico,1,25 Jul 21 a las 04:00,Estadio Akron,Chivas Guadalajara,1,2,Atl. San Luis


In [5]:
results["LocalWin"] = results["GolesVisitante"] < results["GolesLocal"]
# Our "class values"
y_true = results["LocalWin"].values
results.iloc[:5]

,Anio,Torneo,Jornada,Fecha,Lugar,Local,GolesLocal,GolesVisitante,Visitante,LocalWin
0,2022,apertura_mexico,1,23 Jul 21 a las 04:00,Estadio Corregidora,Querétaro,0,0,América,False
1,2022,apertura_mexico,1,24 Jul 21 a las 02:00,Estadio Victoria de Aguascalientes,Necaxa,0,3,Santos Laguna,False
2,2022,apertura_mexico,1,24 Jul 21 a las 04:00,Estadio Olímpico Benito Juárez,FC Juárez,1,3,Toluca,False
3,2022,apertura_mexico,1,24 Jul 21 a las 23:00,Estadio Miguel Hidalgo,Pachuca,4,0,León,True
4,2022,apertura_mexico,1,25 Jul 21 a las 04:00,Estadio Akron,Chivas Guadalajara,1,2,Atl. San Luis,False


In [6]:
print("Porcentaje de ganados de Local: {0:.1f}%".format(100 * results["LocalWin"].sum() / results["LocalWin"].count()))

Porcentaje de ganados de Local: 38.6%


In [7]:
results["LocalLastWin"] = False
results["VisitanteLastWin"] = False
# This creates two new columns, all set to False
results.iloc[:5]

,Anio,Torneo,Jornada,Fecha,Lugar,Local,GolesLocal,GolesVisitante,Visitante,LocalWin,LocalLastWin,VisitanteLastWin
0,2022,apertura_mexico,1,23 Jul 21 a las 04:00,Estadio Corregidora,Querétaro,0,0,América,False,False,False
1,2022,apertura_mexico,1,24 Jul 21 a las 02:00,Estadio Victoria de Aguascalientes,Necaxa,0,3,Santos Laguna,False,False,False
2,2022,apertura_mexico,1,24 Jul 21 a las 04:00,Estadio Olímpico Benito Juárez,FC Juárez,1,3,Toluca,False,False,False
3,2022,apertura_mexico,1,24 Jul 21 a las 23:00,Estadio Miguel Hidalgo,Pachuca,4,0,León,True,False,False
4,2022,apertura_mexico,1,25 Jul 21 a las 04:00,Estadio Akron,Chivas Guadalajara,1,2,Atl. San Luis,False,False,False


In [8]:
# Now compute the actual values for these
# Did the home and visitor teams win their last game?
from collections import defaultdict
won_last = defaultdict(int)

for index, row in results.iterrows():  # Note that this is not efficient
    home_team = row["Local"]
    visitor_team = row["Visitante"]
    row["LocalLastWin"] = won_last[home_team]
    row["VisitanteLastWin"] = won_last[visitor_team]
    
    results.iloc[index] = row
    
    won_last[home_team] = row["LocalWin"]
    won_last[visitor_team] = not row["LocalWin"]
results.iloc[20:25]

,Anio,Torneo,Jornada,Fecha,Lugar,Local,GolesLocal,GolesVisitante,Visitante,LocalWin,LocalLastWin,VisitanteLastWin
20,2022,apertura_mexico,3,07 Ago 21 a las 04:05,Estadio Victoria de Aguascalientes,Necaxa,1,2,Cruz Azul,False,False,True
21,2022,apertura_mexico,3,07 Ago 21 a las 04:06,Estadio Caliente,Tijuana,0,2,Toluca,False,False,True
22,2022,apertura_mexico,3,08 Ago 21 a las 00:00,Estadio Akron,Chivas Guadalajara,2,2,FC Juárez,False,True,False
23,2022,apertura_mexico,3,08 Ago 21 a las 02:00,Estadio Azteca,América,2,0,Puebla,True,True,False
24,2022,apertura_mexico,3,08 Ago 21 a las 04:00,Estadio Universitario de Nuevo León,Tigres UANL,1,1,Santos Laguna,False,False,False


In [9]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=14)

In [10]:
#from sklearn.cross_validation import cross_val_score
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score


# Create a dataset with just the neccessary information
X_previouswins = results[["LocalLastWin", "VisitanteLastWin"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_previouswins, y_true, scoring='accuracy')
print("Using just the last result from the home and visitor teams")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using just the last result from the home and visitor teams
Accuracy: 53.5%


In [11]:
# What about win streaks?
results["LocalWinStreak"] = 0
results["VisitanteWinStreak"] = 0
# Did the home and visitor teams win their last game?
from collections import defaultdict
win_streak = defaultdict(int)

for index, row in results.iterrows():  # Note that this is not efficient
    home_team = row["Local"]
    visitor_team = row["Visitante"]
    row["LocalWinStreak"] = win_streak[home_team]
    row["VisitanteWinStreak"] = win_streak[visitor_team]
    results.iloc[index] = row    
    # Set current win
    if row["LocalWin"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1

In [12]:
clf = DecisionTreeClassifier(random_state=14)
X_winstreak =  results[["LocalLastWin", "VisitanteLastWin", "LocalWinStreak", "VisitanteWinStreak"]].values
scores = cross_val_score(clf, X_winstreak, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
Accuracy: 52.9%


In [13]:
# Let's try see which team is better on the ladder. Using the previous year's ladder
ladder_filename = "./data/tabla_equipos_apertura2022.csv"
ladder = pd.read_csv(ladder_filename)
ladder.iloc[0:18]

,RK,Equipo,Puntos,Jornadas,V,E,D,GF,GC
0,1,América,35,17,10,5,2,21,10
1,2,Atlas Guadalajara,29,17,8,5,4,21,10
2,3,León,29,17,8,5,4,20,14
3,4,Tigres UANL,28,17,7,7,3,26,14
4,5,Santos Laguna,24,17,5,9,3,23,16
5,6,Toluca,24,17,6,6,5,22,22
6,7,Puebla,24,17,6,6,5,16,16
7,8,Cruz Azul,23,17,5,8,4,21,17
8,9,Monterrey,22,17,5,7,5,19,16
9,10,Chivas Guadalajara,22,17,5,7,5,13,13


In [14]:
# We can create a new feature -- HomeTeamRanksHigher\
results["LocalTeamRanksHigher"] = 0
for index, row in results.iterrows():
    home_team = row["Local"]
    visitor_team = row["Visitante"]
    print(ladder[ladder["Equipo"] == home_team]["RK"].values[0])
    home_rank = ladder[ladder["Equipo"] == home_team]["RK"].values[0]
    visitor_rank = ladder[ladder["Equipo"] == visitor_team]["RK"].values[0]
    row["LocalTeamRanksHigher"] = int(home_rank > visitor_rank)
    results.iloc[index] = row
results.iloc[:5]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [15]:
X_homehigher =  results[["LocalLastWin", "VisitanteLastWin", "LocalTeamRanksHigher"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_homehigher, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
Accuracy: 53.5%


In [16]:
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

parameter_space = {
                   "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                   }
clf = DecisionTreeClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_homehigher, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))

Accuracy: 61.4%


In [17]:
# Who won the last match? We ignore home/visitor for this bit
last_match_winner = defaultdict(int)
results["LocalTeamWonLast"] = 0

for index, row in results.iterrows():
    home_team = row["Local"]
    visitor_team = row["Visitante"]
    teams = tuple(sorted([home_team, visitor_team]))  # Sort for a consistent ordering
    # Set in the row, who won the last encounter
    row["LocalTeamWonLast"] = 1 if last_match_winner[teams] == row["Local"] else 0
    results.iloc[index] = row
    # Who won this one?
    winner = row["Local"] if row["LocalWin"] else row["Visitante"]
    last_match_winner[teams] = winner
results.iloc[:5]

,Anio,Torneo,Jornada,Fecha,Lugar,Local,GolesLocal,GolesVisitante,Visitante,LocalWin,LocalLastWin,VisitanteLastWin,LocalWinStreak,VisitanteWinStreak,LocalTeamRanksHigher,LocalTeamWonLast
0,2022,apertura_mexico,1,23 Jul 21 a las 04:00,Estadio Corregidora,Querétaro,0,0,América,False,0,0,0,0,0,0
1,2022,apertura_mexico,1,24 Jul 21 a las 02:00,Estadio Victoria de Aguascalientes,Necaxa,0,3,Santos Laguna,False,0,0,0,0,0,0
2,2022,apertura_mexico,1,24 Jul 21 a las 04:00,Estadio Olímpico Benito Juárez,FC Juárez,1,3,Toluca,False,0,0,0,0,0,0
3,2022,apertura_mexico,1,24 Jul 21 a las 23:00,Estadio Miguel Hidalgo,Pachuca,4,0,León,True,0,0,0,0,0,0
4,2022,apertura_mexico,1,25 Jul 21 a las 04:00,Estadio Akron,Chivas Guadalajara,1,2,Atl. San Luis,False,0,0,0,0,0,0


In [18]:
X_home_higher =  results[["LocalTeamRanksHigher", "LocalTeamWonLast"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_home_higher, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
Accuracy: 61.4%


In [19]:
#from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#encoding = LabelEncoder()
#encoding.fit(results["Home Team"].values)
#home_teams = encoding.transform(results["Home Team"].values)
home_teams=np.asarray(results["Local"].values)
#visitor_teamsl = encoding.transform(results["Visitor Team"].values)
visitor_teams = np.asarray(results["Visitante"].values)
#X_teams = np.vstack([home_teams, visitor_teams]).T
X_teams = np.asarray([home_teams, visitor_teams]).T
onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).todense()

clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 47.1%


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

In [20]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Using full team labels is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

Using full team labels is ranked higher
Accuracy: 49.7%


In [21]:
X_all = np.hstack([X_home_higher, X_teams])
print(X_all.shape)

(153, 38)


In [22]:
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

Using whether the home team is ranked higher
Accuracy: 54.3%


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [31]:
#n_estimators=10, criterion='gini', max_depth=None, 
#min_samples_split=2, min_samples_leaf=1,
#max_features='auto',
#max_leaf_nodes=None, bootstrap=True,
#oob_score=False, n_jobs=1,
#random_state=None, verbose=0, min_density=None, compute_importances=None
parameter_space = {
                   "max_features": [2, 10, 'auto'],
                   "n_estimators": [100,],
                   "criterion": ["gini", "entropy"],
                   "min_samples_leaf": [2, 4, 6],
                   }
clf = RandomForestClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_all, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))
print(grid.best_estimator_)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-

Accuracy: 61.4%
RandomForestClassifier(max_features=2, min_samples_leaf=4, random_state=14)
